# Exercise 1 MLP with PyTorch for CIFAR10

Based on the notebooks provided and discussed during the lecture, set up a notebook for the configuration and training of a MLP on the CIFAR10 images.

[These images](https://www.cs.toronto.edu/~kriz/cifar.html) of size 32x32 pixel show 10 different object categories

Your implementation should provide the following features : 

- Download of the CIFAR10 images form an appropriate server and local storage for further usage.
- Configuration of a custom Dataset (c.f. chapter 6.2.2 in the lecture notes) that will provide the CIFAR10 data to the DataLoader class during training.
- Set up of a SummaryWriter for TensorBoard (c.f. chapter 6.2.3 in the lecture notes) and output of a set of CIFAR10 images (as grid like the figures above) to TensorBoard.
- Set up of a MLP with confiigurable number of hidden layers that takes CIFAR10 (colour) images as input.
- Preparation of the SummaryWriter for continuous output of training and validation loss/error to TensorBoard during the training process.

In [1]:
# ~~~ Imports ~~~
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from torchvision.datasets.cifar import CIFAR10


In [2]:
# ~~~ Transforms ~~~
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# ~~~ Dataset ~~~
class CustomCIFAR10Dataset(CIFAR10):
    def __init__(self, **kwargs):
        cifar10_dataset = CIFAR10(**kwargs)
        self.cifar10_dataset = cifar10_dataset

    def __len__(self):
        return len(self.cifar10_dataset)

    def __getitem__(self, idx):
        data, target = self.cifar10_dataset[idx]
        return data, target

In [8]:
# ~~~ Download CIFAR10 Dataset ~~~
train_dataset = CustomCIFAR10Dataset(root='./data', train=True, download=True, transform=transform)
test_dataset = CustomCIFAR10Dataset(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# ~~~ Parameters ~~~
batch_size = 32
num_epochs = 10
learning_rate = 0.001
num_hidden_layers = 2
hidden_layer_size = 128

In [4]:
# ~~~ Data loaders ~~~
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# ~~~ MLP ~~~
class MLP(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden_layers, hidden_layer_size):
        super(MLP, self).__init__()
        layers = [nn.Flatten()]
        layers.append(nn.Linear(input_size, hidden_layer_size))
        layers.append(nn.ReLU())

        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(hidden_layer_size, hidden_layer_size))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_layer_size, num_classes))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        out = self.layers(x)
        return out


In [6]:
# ~~~ Experience ~~~
model = MLP(3 * 32 * 32, 10, num_hidden_layers, hidden_layer_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [7]:
# ~~~ TensorBoard Setup ~~~
writer = SummaryWriter('runs/cifar10_experiment')

# ~~~ Training ~~~
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Log training information to TensorBoard
    writer.add_scalar('training loss', loss.item(), epoch)

    # Log images to TensorBoard
    img_grid = make_grid(images)
    writer.add_image('CIFAR10 Images', img_grid, epoch)

writer.close()


Epoch [1/10], Step [100/1563], Loss: 1.8642
Epoch [1/10], Step [200/1563], Loss: 1.6989
Epoch [1/10], Step [300/1563], Loss: 1.5546
Epoch [1/10], Step [400/1563], Loss: 1.4690
Epoch [1/10], Step [500/1563], Loss: 1.5200
Epoch [1/10], Step [600/1563], Loss: 1.3329
Epoch [1/10], Step [700/1563], Loss: 1.4116
Epoch [1/10], Step [800/1563], Loss: 1.5245
Epoch [1/10], Step [900/1563], Loss: 1.4949
Epoch [1/10], Step [1000/1563], Loss: 1.8159
Epoch [1/10], Step [1100/1563], Loss: 1.3743
Epoch [1/10], Step [1200/1563], Loss: 1.5110
Epoch [1/10], Step [1300/1563], Loss: 1.4013
Epoch [1/10], Step [1400/1563], Loss: 1.2074
Epoch [1/10], Step [1500/1563], Loss: 1.7704
Epoch [2/10], Step [100/1563], Loss: 1.4996
Epoch [2/10], Step [200/1563], Loss: 1.1926
Epoch [2/10], Step [300/1563], Loss: 1.5976
Epoch [2/10], Step [400/1563], Loss: 1.2378
Epoch [2/10], Step [500/1563], Loss: 1.3722
Epoch [2/10], Step [600/1563], Loss: 1.2517
Epoch [2/10], Step [700/1563], Loss: 1.4453
Epoch [2/10], Step [800/15